In [ ]:
import cv2
import time
import numpy as np
from numpy.random import randint
import random

def gen_pic(path,boxes,j,k):
        
    img=cv2.imread(path)
    if img is None:
        return j,k
    h1,w1,_=img.shape
    if(min(w1,h1)<=210):
        return j,k
    
    num=9
    while(num):
        size=randint(100,min(w1,h1)/2)
        x1=randint(1,w1-size)
        y1=randint(1,h1-size)
        box=np.array([x1,y1,x1+size,y1+size])
        _boxes=boxes.copy()
        if(np.max(IoU(box,np.array(_boxes)))<0.3):
            resize_img=cv2.resize(img[y1:y1+size,x1:x1+size,:],(224,224))
            cv2.imwrite('E:\\friedhelm\\Data\\non-face\\non_face_%d.jpg'%(j),resize_img)            
            j=j+1
            num=num-1

    for bbox in boxes:
        x=max(bbox[0],0)
        y=max(bbox[1],0)
        w=max(bbox[2],0)
        h=max(bbox[3],0)
        num=2 
        while(num):

            size=randint(np.floor(0.5*min(w,h))+1,np.ceil(1.5*max(w,h))+2)
            x1=randint(np.floor(0.5*x)+1,np.ceil(1.5*x)+2)
            y1=randint(np.floor(0.5*y)+1,np.ceil(1.5*y)+2)
            box=np.array([x1,y1,x1+size,y1+size])

            _bbox=np.array(bbox).reshape(1,-1)
            if(IoU(box,_bbox)>0.65):
                resize_img=cv2.resize(img[y1:y1+size,x1:x1+size,:],(224,224))
                cv2.imwrite('E:\\friedhelm\\Data\\face\\face_%d.jpg'%(k),resize_img)            
                k=k+1

                if(random.choice([0,1])):
                    resize_img=cv2.flip(resize_img,1)
                    cv2.imwrite('E:\\friedhelm\\Data\\face\\face_%d.jpg'%(k),resize_img)             
                    k=k+1
                num=num-1  
    return j,k

def IoU(box,boxes):
    
    """
    box为实际，boxes为人脸
    """
    area=boxes[:,2]*boxes[:,3]
    
    x1=np.maximum(box[0],boxes[:,0])
    y1=np.maximum(box[1],boxes[:,1])    
    x2=np.minimum(box[2],boxes[:,2]+boxes[:,0])
    y2=np.minimum(box[3],boxes[:,3]+boxes[:,1]) 
    
    w=np.maximum(0,x2-x1+1)
    h=np.maximum(0,y2-y1+1)
    return w*h/area

def main():
    begin=time.time()
    addr='E:\\friedhelm\\AFLW\\'
    with open(r'E:\friedhelm\alfw.txt') as f:
        j=0
        k=0
        boxes=[]
        path_compare="1"
        for line in f.readlines():
            line=line.strip().split()
            path=addr+line[0]
            x=int(line[1])
            if x<0:x=0
            y=int(line[2])
            if y<0:y=0
            w=int(line[3])
            h=int(line[4])

            if(path_compare==path):
                boxes.append([x,y,w,h])
            else:
                if(path_compare!="1"):
                    j,k=gen_pic(path_compare,boxes,j,k)
                boxes=[]
                path_compare=path
                boxes.append([x,y,w,h])

    print(time.time()-begin) 
    
if __name__=='__main__':
    main()